# |In this notebook we fit a SEIR model to the Moscow Covid-19 data

In [36]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.integrate import odeint
import lmfit
from tqdm.auto import tqdm
from copy import deepcopy

In [37]:
sns.set()
%matplotlib inline

In [38]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
%autoreload 2

In [338]:
from sir_models.seir import SEIR, DayAheadFitter, CurveFitter
from sir_models.utils import stepwise, compute_daily_values, eval_one_day_ahead

# Load data

In [413]:
DATASET_PATH = '/media/boris/ubuntu_data/datasets/covid-19-data/public/data/owid-covid-data.csv'
df = pd.read_csv(DATASET_PATH)
useless_columns = ['iso_code', 'continent',
       'new_vaccinations_smoothed', 'total_vaccinations_per_hundred',
       'people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred',
       'new_vaccinations_smoothed_per_million', 'stringency_index',
       'population_density', 'median_age', 'aged_65_older',
       'aged_70_older', 'gdp_per_capita', 'extreme_poverty',
       'cardiovasc_death_rate', 'diabetes_prevalence', 'female_smokers',
       'male_smokers', 'handwashing_facilities', 'hospital_beds_per_thousand',
       'life_expectancy', 'human_development_index', 'total_cases_per_million', 'new_cases_per_million',
       'new_cases_smoothed_per_million', 'total_deaths_per_million',
       'new_deaths_per_million', 'new_deaths_smoothed_per_million', 'icu_patients_per_million', 'hosp_patients_per_million', 'weekly_icu_admissions_per_million',
        'weekly_hosp_admissions_per_million', 'total_tests_per_thousand',  'new_tests_per_thousand', 'new_tests_smoothed_per_thousand', 'icu_patients', 'hosp_patients',
       'weekly_icu_admissions', 'weekly_hosp_admissions',
                   'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'new_vaccinations',
                  ]
df = df.drop(columns=useless_columns)
df.date = pd.to_datetime(df.date)

df = df[df.location == 'Russia']

df['total_deaths'] = df.total_deaths.fillna(0)
df['new_deaths'] = df.new_deaths.fillna(0)
df.head().T

,52406,52407,52408,52409,52410
location,Russia,Russia,Russia,Russia,Russia
date,2020-01-31 00:00:00,2020-02-01 00:00:00,2020-02-02 00:00:00,2020-02-03 00:00:00,2020-02-04 00:00:00
total_cases,2,2,2,2,2
new_cases,2,0,0,0,0
new_cases_smoothed,NaN,NaN,NaN,NaN,NaN
total_deaths,0,0,0,0,0
new_deaths,0,0,0,0,0
new_deaths_smoothed,NaN,NaN,NaN,NaN,NaN
reproduction_rate,NaN,NaN,NaN,NaN,NaN
new_tests,NaN,NaN,NaN,NaN,NaN


# Define model and fitter

In [420]:
class OWIDCurveFitter(CurveFitter):
    def get_initial_conditions(self, model, data):
        # Simulate such initial params as to obtain as many deaths as in data
        sus_population = model.params['sus_population']
        
        new_params = deepcopy(model.params)
        for key, value in new_params.items():
            if key.startswith('t'):
                new_params[key].value = 0
        new_model = SEIR(params=new_params)

        t = np.arange(365)
        (S, E, I, R, D), history = new_model.predict(t, (sus_population - 1, 0, 1, 0, 0), history=False)
        fatality_day = np.argmax(D >= data.total_deaths.fillna(0).iloc[0])

        I0 = I[fatality_day]
        E0 = E[fatality_day]
        Rec0 = R[fatality_day]
        D0 = D[fatality_day]
        S0 = S[fatality_day]
        return (S0, E0, I0, Rec0, D0)
    
    def residual(self, params, t_vals, data, model):
        model.params = params

        initial_conditions = self.get_initial_conditions(model, data)

        (S, E, I, R, D), history = model.predict(t_vals, initial_conditions, history=True)
        new_exposed, new_infected, new_recovered, new_dead = compute_daily_values(S, E, I, R, D)
        cumulative_infected = I.cumsum()
        true_daily_cases = data['new_cases'][:len(new_infected)].fillna(0)
        true_daily_deaths = data['new_deaths'][:len(new_dead)].fillna(0)
        
        resid_I_total = (cumulative_infected - data['total_cases']) / (np.maximum(cumulative_infected, data['total_cases']) + 1e-10)
        resid_I_new = (new_infected - true_daily_cases) / (np.maximum(new_infected, true_daily_cases) + 1e-10)
        
        resid_D_total = (D - data['total_deaths']) / (np.maximum(D, data['total_deaths']) + 1e-10)
        resid_D_new = (new_dead - true_daily_deaths) / (np.maximum(new_dead, true_daily_deaths) + 1e-10)
        
        residuals = np.concatenate([
                                    resid_I_total, 
                                    resid_I_new,
                                    resid_D_total,
                                    resid_D_new,
                                   ]).flatten()
        return residuals

In [466]:
class SEIR_OWID(SEIR):
    def get_fit_params(self, data):
        params = lmfit.Parameters()
        # Non-variable
        params.add("base_population", value=df.iloc[0].population, vary=False)
        params.add("pre_existing_immunity", value=0.1806, vary=False)
        params.add("sus_population", expr='base_population - base_population * pre_existing_immunity', vary=False)

        # Variable
        params.add("r0", value=3.5, min=2.5, max=4, vary=False)
        params.add("delta", value=1 / 5.15, min=1/8, max=1/3, vary=True)  # E -> I rate
        params.add("gamma", value=1 / 9.5, min=1/20, max=1/2,  vary=True)  # I -> R rate
        params.add("rho", value=1 / 14, min=1/20, max=1/7, vary=False)  # I -> D rate

        params.add("alpha", value=0.0066, min=0.0001, max=0.05, vary=False) # Probability to die if infected

        params.add(f"t0_q", value=0, min=0, max=0.99, brute_step=0.1, vary=True)
        piece_size = self.stepwise_size
        for t in range(piece_size, len(data), piece_size):
            params.add(f"t{t}_q", value=0.5, min=0, max=0.99, brute_step=0.1, vary=True)

        return params


# Model

In [467]:
train_subset = df[
#                     (df.date >= '2020-03-25') & 
                  (df.date <= '2020-11-30')]
train_subset.head()

,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,reproduction_rate,new_tests,total_tests,new_tests_smoothed,positive_rate,tests_per_case,tests_units,population
52406,Russia,2020-01-31,2.0,2.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145934460.0
52407,Russia,2020-02-01,2.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145934460.0
52408,Russia,2020-02-02,2.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145934460.0
52409,Russia,2020-02-03,2.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145934460.0
52410,Russia,2020-02-04,2.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145934460.0


In [468]:
test_subset = df[df.date > train_subset.iloc[-1].date]
test_subset.date[:3]

52711   2020-12-01
52712   2020-12-02
52713   2020-12-03
Name: date, dtype: datetime64[ns]

In [469]:
model = SEIR_OWID(stepwise_size=30)
fitter = OWIDCurveFitter(use_recovered=False)
fitter.fit(model, train_subset)

Iter 0 | MAE: 0.8348
Iter 10 | MAE: 0.8348
Iter 20 | MAE: 0.8348
Iter 30 | MAE: 0.8348
Iter 40 | MAE: 0.8348
Iter 50 | MAE: 0.8348
Iter 60 | MAE: 0.8348
Iter 70 | MAE: 0.8347
Iter 80 | MAE: 0.8347
Iter 90 | MAE: 0.8346
Iter 100 | MAE: 0.8346
Iter 110 | MAE: 0.8345
Iter 120 | MAE: 0.8345
Iter 130 | MAE: 0.8345
Iter 140 | MAE: 0.8344
Iter 150 | MAE: 0.8342
Iter 160 | MAE: 0.8341
Iter 170 | MAE: 0.8341
Iter 180 | MAE: 0.8339
Iter 190 | MAE: 0.8336
Iter 200 | MAE: 0.8336
Iter 210 | MAE: 0.8330
Iter 220 | MAE: 0.8317
Iter 230 | MAE: 0.8307
Iter 240 | MAE: 0.8307
Iter 250 | MAE: 0.8285
Iter 260 | MAE: 0.8227
Iter 270 | MAE: 0.8227
Iter 280 | MAE: 0.8159
Iter 290 | MAE: 0.8112
Iter 300 | MAE: 0.8062
Iter 310 | MAE: 0.8062
Iter 320 | MAE: 0.8040
Iter 330 | MAE: 0.8100
Iter 340 | MAE: 0.8030
Iter 350 | MAE: 0.7992
Iter 360 | MAE: 0.7939
Iter 370 | MAE: 0.7939
Iter 380 | MAE: 0.7938
Iter 390 | MAE: 0.7896
Iter 400 | MAE: 0.7896
Iter 410 | MAE: 0.7872
Iter 420 | MAE: 0.7846
Iter 430 | MAE: 0.7822

In [ ]:
result = fitter.result
result

In [ ]:
train_initial_conditions = fitter.get_initial_conditions(model, train_subset)
train_t = np.arange(len(train_subset))

(S, E, I, R, D), history = model.predict(train_t, train_initial_conditions)
new_exposed, new_infected, new_recovered, new_dead = compute_daily_values(S, E, I, R, D)

In [ ]:
plt.figure()
history.rt.plot()
history.beta.plot()
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(train_subset.date, I, label='infected')
plt.plot(train_subset.date, E, label='exposed')
plt.plot(train_subset.date, D, label='dead')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(new_infected, label='daily infected')
plt.plot(new_dead, label='daily deaths')
plt.plot(new_recovered, label='daily recovered')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(train_subset.date, train_subset['total_deaths'], label='ground truth')
plt.plot(train_subset.date, D, label='predicted')
plt.legend()
plt.title('Total deaths')
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(train_subset.date, train_subset['new_deaths'], label='ground truth')
plt.plot(train_subset.date[:-1], new_dead, label='predicted')
plt.legend()
plt.title('Daily deaths')
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(train_subset.date, train_subset['total_cases'], label='ground truth')
plt.plot(train_subset.date, I.cumsum(), label='predicted')
plt.legend()
plt.title('Total cases')
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(train_subset.date, train_subset['new_cases'], label='ground truth')
plt.plot(train_subset.date[:-1], new_infected, label='predicted')
plt.legend()
plt.title('Daily new cases')
plt.show()

# Obtain forecast

In [ ]:
test_t = len(train_subset) + np.arange(len(test_subset))

In [ ]:
train_t, test_t

In [ ]:
test_initial_conds = (S[-1], E[-1], I[-1], R[-1], D[-1])

In [ ]:
(test_S, test_E, test_I, test_R, test_D), history = model.predict(test_t, test_initial_conds)

In [ ]:
test_new_exposed, test_new_infected, test_new_recovered, test_new_dead = compute_daily_values(test_S, test_E, test_I, test_R, test_D)

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(train_subset.date, train_subset['total_deaths'], label='train ground truth')
plt.plot(train_subset.date, D, label='train fit')

plt.plot(test_subset.date, test_subset['total_deaths'], label='test ground truth', color='red')
plt.plot(test_subset.date, test_D, label='test forecasted', color='red', linestyle=':')
plt.legend()
plt.title('Total deaths')
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(train_subset.date, train_subset['new_cases'], label='train ground truth')
plt.plot(train_subset.date[:-1], new_infected, label='train fit')

plt.plot(test_subset.date, test_subset['new_cases'], label='test ground truth', color='red')
plt.plot(test_subset.date[:-1], test_new_infected, label='test forecasted', color='red', linestyle=':')
plt.legend()
plt.title('New daily cases')
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(train_subset.date, train_subset['total_cases'], label='train ground truth')
plt.plot(train_subset.date, I.cumsum(), label='train fit')

plt.plot(test_subset.date, test_subset['total_cases'], label='test ground truth', color='red')
plt.plot(test_subset.date, I.sum()+test_I.cumsum(), label='test forecasted', color='red', linestyle=':')
plt.legend()
plt.title('Total Infections')
plt.show()

# 1-day ahead evaluate

In [ ]:
eval_one_day_ahead(df, SEIR, CurveFitter, eval_period_start='2020-05-01', n_eval_points=50)